In [ ]:
!pip install -U transformers datasets huggingface_hub accelerate transformers[torch] evaluate peft --quiet

from datasets import load_dataset, concatenate_datasets, Dataset, ClassLabel, load_from_disk, load_metric
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer, AutoTokenizer
import numpy as np
import matplotlib.pyplot as plt
import accelerate
import transformers

In [ ]:
output_dir = "/content/drive/MyDrive/Spam detection/Model/checkpoint-12748"

tokenizer = AutoTokenizer.from_pretrained(output_dir)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(output_dir, num_labels=2)

In [ ]:
model.eval()

In [ ]:
import torch

examples = [
    {'text': 'این تست متن فارسی غیر اسپم است.', 'label': 0},
    {'text': 'خرید هاست لینوکس ایران', 'label': 1},
    {'text': 'ترجمه تخصصی و فنی به آلمانی: رویکردها و استراتژی‌ها', 'label': 0},
    {'text': 'فایلهای پرکاربرد و آموزشی و با کیفیت برای استفاده دانشجویان وتحقیقات علمی و پژوهشی', 'label': 1},
    {'text': 'فرصت‌های تحصیل رایگان در اروپا: بورس‌های تحصیلی به عنوان پلی به دسترسی به تعلیمات برتر', 'label': 1},
    {'text': 'امروز ساعت 6:45 از خواب بیدار شدم و از همون لحظه حوصله هیچکس رو ندارم :|', 'label': 0},
    {'text': '''اَلا یا اَیُّهَا السّاقی اَدِرْ کَأسَاً و ناوِلْها که عشق آسان نمود اوّل ولی افتاد مشکل‌ه''', 'label': 0},
    {'text': 'سخنگوی شورای امنیت ملی کاخ سفید گفت که این کشور به ایران پیام داده که نمی‌خواهد شاهد گسترش درگیری در منطقه باشد.', 'label': 0},
    {'text': 'سوالات استخدامی علوم پزشکی و بیمارستانها 1402 ,سوالات استخدامی وزارت بهداشت+سوالات استخدامی بیمارستان 1402- نمونه سوالات استخدامی رایگان,', 'label': 1},
    {'text': ' روند رانندگی بی صدا، بدون سر و صدای غیر عادی، با ظاهری بسیارشیک و جمع و جور است و دید راننده را محدود نمی کند. توربین این دستگاه در طول روز برای پخش عود می چرخد ترکیبی قوی از آلیاژ با مقاومت بالا و سرامیک طبیعی، بدون ترس از قرار گرفتن در معرض آفتاب و در تابستان بسیار قوی کار میکند.قیمت این محصول...تومان', 'label': 1},
    {'text': 'DeciLM-7B: The Fastest and Most Accurate 7B-Parameter LLM to Date', 'label': 0},
    {'text': 'Telecom Industry Is Mad Because the FCC Might Examine High Broadband Prices', 'label': 0},
    {'text': 'Well, here is hope that this will be a first step in bringing US internet access to at least something comparable to Balkans. ', 'label': 0},
    {'text': 'Agree to notifications to allow news feed', 'label': 1},
    {'text': '6 Ways to Boost Your Coffee with Vitamins and Antioxidants', 'label': 1},
    {'text': 'The section is not fundamental to understand the rest. It explains in brief how numbers are represented in computers. Since quantization is about going from one representation to another, it can be useful to have some basics, but it is definitely not mandatory.', 'label': 0},
    {'text': 'پیراهن پاییزه مردانه مدل Sitro (سرمه ای)', 'label': 1},
    {'text': 'دانلود انیمه آکوداما درایو با زیرنویس فارسی بدون سانسور رایگان', 'label': 1},
    {'text': 'منتظرالمهدی۳۱۳', 'label': 0},
    {'text': 'ارتش پاکستان در پاسخ به حملات موشکی جمهوری اسلامی ایران به خاک خود، نقاطی را در خاک ایران مورد حمله موشکی قرار داد.', 'label': 0}
]

for example in examples:
  print('*' * 40)
  print('Text:', example['text'], '\n')

  inputs = tokenizer(example['text'], return_tensors="pt")
  inputs = {name: tensor for name, tensor in inputs.items()}

  with torch.no_grad():
    logits = model(**inputs).logits

  print('Ref:', {0: 'Ham', 1: 'Spam'}[example['label']])
  print('Model:', {0: 'Ham', 1: 'Spam'}[logits.argmax().item()])
  print('Ham confidence:', logits.softmax(-1)[0][0].item())
  print('Spam confidence:', logits.softmax(-1)[0][1].item())

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

In [ ]:
import os


def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

In [ ]:
import torch

examples = [
    {'text': 'این تست متن فارسی غیر اسپم است.', 'label': 0},
    {'text': 'خرید هاست لینوکس ایران', 'label': 1},
    {'text': 'ترجمه تخصصی و فنی به آلمانی: رویکردها و استراتژی‌ها', 'label': 0},
    {'text': 'فایلهای پرکاربرد و آموزشی و با کیفیت برای استفاده دانشجویان وتحقیقات علمی و پژوهشی', 'label': 1},
    {'text': 'فرصت‌های تحصیل رایگان در اروپا: بورس‌های تحصیلی به عنوان پلی به دسترسی به تعلیمات برتر', 'label': 1},
    {'text': 'امروز ساعت 6:45 از خواب بیدار شدم و از همون لحظه حوصله هیچکس رو ندارم :|', 'label': 0},
    {'text': '''اَلا یا اَیُّهَا السّاقی اَدِرْ کَأسَاً و ناوِلْها که عشق آسان نمود اوّل ولی افتاد مشکل‌ه''', 'label': 0},
    {'text': 'سخنگوی شورای امنیت ملی کاخ سفید گفت که این کشور به ایران پیام داده که نمی‌خواهد شاهد گسترش درگیری در منطقه باشد.', 'label': 0},
    {'text': 'سوالات استخدامی علوم پزشکی و بیمارستانها 1402 ,سوالات استخدامی وزارت بهداشت+سوالات استخدامی بیمارستان 1402- نمونه سوالات استخدامی رایگان,', 'label': 1},
    {'text': ' روند رانندگی بی صدا، بدون سر و صدای غیر عادی، با ظاهری بسیارشیک و جمع و جور است و دید راننده را محدود نمی کند. توربین این دستگاه در طول روز برای پخش عود می چرخد ترکیبی قوی از آلیاژ با مقاومت بالا و سرامیک طبیعی، بدون ترس از قرار گرفتن در معرض آفتاب و در تابستان بسیار قوی کار میکند.قیمت این محصول...تومان', 'label': 1},
    {'text': 'DeciLM-7B: The Fastest and Most Accurate 7B-Parameter LLM to Date', 'label': 0},
    {'text': 'Telecom Industry Is Mad Because the FCC Might Examine High Broadband Prices', 'label': 0},
    {'text': 'Well, here is hope that this will be a first step in bringing US internet access to at least something comparable to Balkans. ', 'label': 0},
    {'text': 'Agree to notifications to allow news feed', 'label': 1},
    {'text': '6 Ways to Boost Your Coffee with Vitamins and Antioxidants', 'label': 1},
    {'text': 'The section is not fundamental to understand the rest. It explains in brief how numbers are represented in computers. Since quantization is about going from one representation to another, it can be useful to have some basics, but it is definitely not mandatory.', 'label': 0},
    {'text': 'پیراهن پاییزه مردانه مدل Sitro (سرمه ای)', 'label': 1},
    {'text': 'دانلود انیمه آکوداما درایو با زیرنویس فارسی بدون سانسور رایگان', 'label': 1},
    {'text': 'منتظرالمهدی۳۱۳', 'label': 0},
    {'text': 'ارتش پاکستان در پاسخ به حملات موشکی جمهوری اسلامی ایران به خاک خود، نقاطی را در خاک ایران مورد حمله موشکی قرار داد.', 'label': 0}
]

for example in examples:
  print('*' * 40)
  print('Text:', example['text'], '\n')

  inputs = tokenizer(example['text'], return_tensors="pt")
  inputs = {name: tensor for name, tensor in inputs.items()}

  with torch.no_grad():
    logits = quantized_model(**inputs).logits

  print('Ref:', {0: 'Ham', 1: 'Spam'}[example['label']])
  print('Model:', {0: 'Ham', 1: 'Spam'}[logits.argmax().item()])
  print('Ham confidence:', logits.softmax(-1)[0][0].item())
  print('Spam confidence:', logits.softmax(-1)[0][1].item())